# 카카오 API 인증 스크립트

이 스크립트는 카카오 API 인증을 처리하며, 액세스 토큰을 생성하고 갱신하는 기능을 포함합니다. 토큰을 로컬에 저장하여 재사용할 수 있으며, 이를 통해 반복적인 인증 과정을 줄일 수 있습니다.

## 전체 코드 실행 흐름
1. **토큰 로드**: 기존에 저장된 토큰이 있는지 확인하고 로드합니다.
2. **조건부 갱신**: 토큰이 만료되었는지 확인하여 만료된 경우 갱신합니다.
3. **새 토큰 발급**: 토큰이 없거나 만료되지 않은 경우 새로 발급받습니다.
4. **토큰 저장**: 발급받은 토큰을 로컬 파일에 저장합니다.

## 모듈 및 함수 설명

### 임포트 구문과 서비스 키
- **from PyKakao import Message**: PyKakao 라이브러리에서 `Message` 클래스를 임포트합니다.
- **import os**: 운영 체제와 상호작용할 수 있는 함수들을 제공합니다.
- **import json**: JSON 데이터를 다룰 수 있게 합니다.
- **import time**: 시간 관련 함수를 사용할 수 있게 합니다.

### 서비스 키 가져오기
- 환경 변수에서 서비스 키를 가져옵니다. 이는 보안을 위해 코드에 직접 키를 쓰지 않고 환경 변수로 관리합니다.

### 토큰을 저장할 파일 경로
- 토큰을 저장할 파일 경로를 지정합니다.

### Message 인스턴스 생성
- `Message` 클래스의 인스턴스를 생성합니다. 이를 통해 API 호출을 할 수 있습니다.

### 토큰 저장 함수
- **save_tokens(tokens)**: 토큰을 파일에 저장하는 함수입니다.
    - 토큰 발급 시각을 추가합니다.
    - JSON 형식으로 파일에 저장합니다.

### 토큰 로드 함수
- **load_tokens()**: 파일에서 토큰을 로드하는 함수입니다.
    - 파일이 존재하면 JSON 형식으로 로드하고, 형식이 잘못되었으면 None을 반환합니다.
    - 파일이 없으면 None을 반환합니다.

### 새로운 토큰 생성 함수
- **get_new_tokens()**: 새로운 액세스 토큰을 생성하는 함수입니다.
    - 인증 URL을 생성하고 사용자에게 표시합니다.
    - 사용자가 인증 코드를 입력하면 액세스 토큰을 발급받습니다.
    - 반환된 토큰 데이터를 확인하고 저장합니다.

### 액세스 토큰 갱신 함수
- **refresh_access_token(refresh_token)**: 액세스 토큰을 갱신하는 함수입니다.
    - 리프레시 토큰을 사용하여 새로운 액세스 토큰을 발급받습니다.
    - 반환된 토큰 데이터를 확인하고 저장합니다.

### 토큰 로드 및 조건부 갱신
- **조건부 갱신 로직**: 토큰을 로드한 후 조건에 따라 갱신합니다.
  - 토큰이 존재하고 30일 이상 경과한 경우 갱신합니다.
  - 토큰이 존재하지 않거나 만료되지 않은 경우 저장된 토큰을 사용합니다.
  - 토큰이 없는 경우 새로 발급받습니다.

### 인증 URL 생성 및 출력
- **get_new_tokens()**: 새로운 토큰을 발급받는 함수입니다.
  - 인증 URL을 생성하고 출력하여 사용자가 인증 코드를 받을 수 있도록 합니다.
  - 사용자로부터 입력받은 인증 코드를 사용하여 액세스 토큰을 발급받습니다.
  - 발급된 토큰 데이터를 확인하고 저장합니다.

### 액세스 토큰 갱신
- **refresh_access_token(refresh_token)**: 기존 리프레시 토큰을 사용하여 액세스 토큰을 갱신하는 함수입니다.
  - 리프레시 토큰을 사용하여 새 액세스 토큰을 발급받고, 반환된 토큰 데이터를 확인하여 저장합니다.

In [1]:
from PyKakao import Message
import os
import json
import time

In [2]:
# 서비스 키 가져오기
service_key = os.environ.get("KAKAO_REST_API")

# 토큰을 저장할 파일 경로
TOKEN_FILE = '../json/kakao_tokens.json'

# Message 인스턴스 생성
API = Message(service_key=service_key)

In [3]:
# 토큰 저장 함수
def save_tokens(tokens):
    tokens['timestamp'] = time.time()  # 토큰 발급 시각 저장
    with open(TOKEN_FILE, 'w') as file:
        json.dump(tokens, file)

In [4]:
# 토큰 로드 함수
def load_tokens():
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'r') as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                # 파일이 비어있거나 JSON 형식이 잘못된 경우 None 반환
                return None
    return None

In [5]:
# 새로운 토큰 생성 함수
def get_new_tokens():
    # 인증 URL 생성 및 출력
    auth_url = API.get_url_for_generating_code()
    print("인증 URL:", auth_url)
    
    # 여기서 auth_url을 통해 인증 코드를 받습니다.
    auth_code = input("인증 코드를 입력하세요: ")

    # 액세스 토큰 발급
    tokens = API.get_access_token_by_redirected_url(auth_code)
    
    # 반환된 값 확인
    print("반환된 토큰 데이터:", tokens)

    # tokens가 문자열인지 확인하고, 그렇다면 JSON으로 변환하지 않음
    if not isinstance(tokens, dict):
        tokens = {'access_token': tokens, 'timestamp': time.time()}

    save_tokens(tokens)
    return tokens

In [6]:
# 액세스 토큰 갱신 함수
def refresh_access_token(refresh_token):
    tokens = API.refresh_access_token(refresh_token)
    
    # 반환된 값 확인
    print("반환된 토큰 데이터:", tokens)

    # tokens가 문자열인지 확인하고, 그렇다면 JSON으로 변환하지 않음
    if not isinstance(tokens, dict):
        tokens = {'access_token': tokens, 'timestamp': time.time()}
    
    save_tokens(tokens)
    return tokens

In [7]:
# 토큰 로드
tokens = load_tokens()

if tokens:
    # 토큰 유효성 검사
    expiration_time = 21600  # 6시간 = 21600초
    token_age = time.time() - tokens.get('timestamp', 0)

    if token_age > expiration_time:
        # 액세스 토큰 갱신
        print("액세스 토큰 갱신 중...")
        tokens = refresh_access_token(tokens.get('access_token'))
else:
    # 새로운 토큰 발급
    tokens = get_new_tokens()

인증 URL: https://accounts.kakao.com/login?continue=https%3A%2F%2Fkauth.kakao.com%2Foauth%2Fauthorize%3Fscope%3Dtalk_message%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Flocalhost%253A5000%26through_account%3Dtrue%26client_id%3Da564118b6791fccfe052e927d15314bc
인증 코드를 입력하세요: https://localhost:5000/?code=nQtBgD3YicbNk8kwG_ELWdLvxmSVnHQzKnSzjEnxusYHieOxW5n4KwAAAAQKKcjYAAABj80UAHSBPKUF0hG4dQ
반환된 토큰 데이터: iI2ZZZ5POtQb_9RtPobNIFryhzV4FYneAAAAAQorDKgAAAGPzRSQFFv0-avl6D9k


## 아래부터는 메시지 전송 테스트 코드

In [8]:
if tokens:
    # 액세스 토큰 설정
    API.set_access_token(tokens['access_token'])

    # 메시지 유형 - 피드
    message_type = "feed"

    # 파라미터
    content = {
        "title": "오늘의 디저트",
        "description": "아메리카노, 빵, 케익",
        "image_url": "https://mud-kage.kakao.com/dn/NTmhS/btqfEUdFAUf/FjKzkZsnoeE4o19klTOVI1/openlink_640x640s.jpg",
        "image_width": 640,
        "image_height": 640,
        "link": {
            "web_url": "http://www.daum.net",
            "mobile_web_url": "http://m.daum.net",
            "android_execution_params": "contentId=100",
            "ios_execution_params": "contentId=100"
        }
    }

    item_content = {
        "profile_text": "Kakao",
        "profile_image_url": "https://mud-kage.kakao.com/dn/Q2iNx/btqgeRgV54P/VLdBs9cvyn8BJXB3o7N8UK/kakaolink40_original.png",
        "title_image_url": "https://mud-kage.kakao.com/dn/Q2iNx/btqgeRgV54P/VLdBs9cvyn8BJXB3o7N8UK/kakaolink40_original.png",
        "title_image_text": "Cheese cake",
        "title_image_category": "Cake",
        "items": [
            {"item": "Cake1", "item_op": "1000원"},
            {"item": "Cake2", "item_op": "2000원"},
            {"item": "Cake3", "item_op": "3000원"},
            {"item": "Cake4", "item_op": "4000원"},
            {"item": "Cake5", "item_op": "5000원"}
        ],
        "sum": "Total",
        "sum_op": "15000원"
    }

    social = {
        "like_count": 100,
        "comment_count": 200,
        "shared_count": 300,
        "view_count": 400,
        "subscriber_count": 500
    }

    buttons = [
        {
            "title": "웹으로 이동",
            "link": {
                "web_url": "http://www.daum.net",
                "mobile_web_url": "http://m.daum.net"
            }
        },
        {
            "title": "앱으로 이동",
            "link": {
                "android_execution_params": "contentId=100",
                "ios_execution_params": "contentId=100"
            }
        }
    ]

    API.send_message_to_me(
        message_type=message_type,
        content=content, 
        item_content=item_content, 
        social=social, 
        buttons=buttons
    )
else:
    print("토큰 발급 실패.")

액세스 토큰 설정 완료
메시지 전송 성공


In [9]:
if tokens:
    # 액세스 토큰 설정
    API.set_access_token(tokens['access_token'])

    # 메시지 유형 - 리스트
    message_type = "list"

    # 파라미터
    header_title = "WEEKELY MAGAZINE"
    header_link = {
                "web_url": "http://www.daum.net",
                "mobile_web_url": "http://m.daum.net",
                "android_execution_params": "main",
                "ios_execution_params": "main"
            }
    contents = [
                {
                    "title": "자전거 라이더를 위한 공간",
                    "description": "매거진",
                    "image_url": "https://mud-kage.kakao.com/dn/QNvGY/btqfD0SKT9m/k4KUlb1m0dKPHxGV8WbIK1/openlink_640x640s.jpg",
                    "image_width": 640,
                    "image_height": 640,
                    "link": {
                        "web_url": "http://www.daum.net/contents/1",
                        "mobile_web_url": "http://m.daum.net/contents/1",
                        "android_execution_params": "/contents/1",
                        "ios_execution_params": "/contents/1"
                    }
                },
                {
                    "title": "비쥬얼이 끝내주는 오레오 카푸치노",
                    "description": "매거진",
                    "image_url": "https://mud-kage.kakao.com/dn/boVWEm/btqfFGlOpJB/mKsq9z6U2Xpms3NztZgiD1/openlink_640x640s.jpg",
                    "image_width": 640,
                    "image_height": 640,
                    "link": {
                        "web_url": "http://www.daum.net/contents/2",
                        "mobile_web_url": "http://m.daum.net/contents/2",
                        "android_execution_params": "/contents/2",
                        "ios_execution_params": "/contents/2"
                    }
                },
                {
                    "title": "감성이 가득한 분위기",
                    "description": "매거진",
                    "image_url": "https://mud-kage.kakao.com/dn/NTmhS/btqfEUdFAUf/FjKzkZsnoeE4o19klTOVI1/openlink_640x640s.jpg",
                    "image_width": 640,
                    "image_height": 640,
                    "link": {
                        "web_url": "http://www.daum.net/contents/3",
                        "mobile_web_url": "http://m.daum.net/contents/3",
                        "android_execution_params": "/contents/3",
                        "ios_execution_params": "/contents/3"
                    }
                }
            ]
    buttons = [
                {
                    "title": "웹으로 이동",
                    "link": {
                        "web_url": "http://www.daum.net",
                        "mobile_web_url": "http://m.daum.net"
                    }
                },
                {
                    "title": "앱으로 이동",
                    "link": {
                        "android_execution_params": "main",
                        "ios_execution_params": "main"
                    }
                }
            ]


    API.send_message_to_me(
        message_type=message_type,
        header_title=header_title, 
        header_link=header_link, 
        contents=contents, 
        buttons=buttons,
        )
else:
    print("토큰 발급 실패.")

액세스 토큰 설정 완료
메시지 전송 성공


In [10]:
if tokens:
    # 액세스 토큰 설정
    API.set_access_token(tokens['access_token'])

    # 메시지 유형 - 위치
    message_type = "location"

    # 파라미터
    address = "경기 성남시 분당구 판교역로 235 에이치스퀘어 N동 7층"
    address_title = "카카오 판교오피스"
    content = {
                    "title": "카카오 판교오피스",
                    "description": "카카오 판교오피스 위치입니다.",
                    "image_url": "https://mud-kage.kakao.com/dn/drTdbB/bWYf06POFPf/owUHIt7K7NoGD0hrzFLeW0/kakaolink40_original.png",
                    "image_width": 800,
                    "image_height": 800,
                    "link": {
                        "web_url": "https://developers.kakao.com",
                        "mobile_web_url": "https://developers.kakao.com/mobile",
                        "android_execution_params": "platform=android",
                        "ios_execution_params": "platform=ios"
                    }
                }
    buttons = [
                    {
                        "title": "웹으로 보기",
                        "link": {
                            "web_url": "https://developers.kakao.com",
                            "mobile_web_url": "https://developers.kakao.com/mobile"
                        }
                    }
                ]

    API.send_message_to_me(
        message_type=message_type,
        address=address, 
        address_title=address_title, 
        content=content, 
        buttons=buttons,
        )
else:
    print("토큰 발급 실패.")

액세스 토큰 설정 완료
메시지 전송 성공
